In [1]:
import numpy as np
import cv2
#Import necessary functions
from matplotlib import pyplot as plt
import skimage.io
import skimage.color
from planarH import computeH_ransac, computeH_ransac_adaptive, warped, compositeH, computeH_norm


#Write script for Q14
img1 = cv2.imread('../data/image1.png')
img2 = cv2.imread('../data/image2.png')
img3 = cv2.imread('../data/image3.png')
img4 = cv2.imread('../data/image4.png')


In [ ]:
# First, find the order of the given images
def find_image_sequence(images):
    # Compute pairwise image similarities using mean squared error (MSE)
    similarities = []
    for i in range(len(images)):
        mse_values = []
        for j in range(len(images)):
            if i != j:
                mse = np.mean((images[i].astype("float") - images[j].astype("float")) ** 2)
                mse_values.append(mse)
        similarities.append(mse_values)
    
    # Determine the order of images based on similarity scores
    image_sequence = []
    used_indices = []
    for i in range(len(similarities)):
        min_mse = np.inf
        min_idx = -1
        for j in range(len(similarities)):
            if i != j and j not in used_indices:
                if similarities[i][j] < min_mse:
                    min_mse = similarities[i][j]
                    min_idx = j
        if min_idx != -1:
            image_sequence.append(i)
            used_indices.append(min_idx)
    
    return image_sequence

In [ ]:
def findHomography(kp1, kp2, matches):

    matches = sorted(matches, key=lambda x: x.distance)
    num_matches = int(len(matches) * 0.5)  # choose 10% of total matches
    best_matches = matches[:num_matches]
    pts1_best = np.float32([kp1[m.queryIdx].pt for m in best_matches]).reshape(-1, 2)
    pts2_best = np.float32([kp2[m.trainIdx].pt for m in best_matches]).reshape(-1, 2)
    H2to1, inliers = computeH_ransac_adaptive(pts2_best, pts1_best, 300, 2)
    H2to1 = computeH_norm(pts2_best[inliers], pts1_best[inliers])

    return H2to1

In [ ]:
orb = cv2.ORB_create(nfeatures=40000,
    scaleFactor=1.2,
    nlevels=8,
    edgeThreshold=31,
    firstLevel=0,
    WTA_K=2,
    scoreType=cv2.ORB_HARRIS_SCORE,
    patchSize=31,
    fastThreshold=20,)

kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)
kp3, des3 = orb.detectAndCompute(img3, None)
kp4, des4 = orb.detectAndCompute(img4, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors of reference image with other images
matches_2to1 = bf.match(des1, des2)
matches_3to1 = bf.match(des1, des3)
matches_4to1 = bf.match(des1, des4)


H2to1 = findHomography(kp1, kp2, matches_2to1)
H3to1 = findHomography(kp1, kp3, matches_3to1)
H4to1 = findHomography(kp1, kp4, matches_4to1)

# Select the image with the smallest total distance as the neighboring image
total_distance = []
total_distance.append(sum([m.distance for m in matches_2to1]))
total_distance.append(sum([m.distance for m in matches_3to1]))
total_distance.append(sum([m.distance for m in matches_4to1]))

neighbor_idx = total_distance.index(min(total_distance))

if neighbor_idx == 0:
    neighboring_image = img2
elif neighbor_idx == 1:
    neighboring_image = img3
else:
    neighboring_image = img4

# Use homography to warp neighboring image to the reference image's coordinate system
# and concatenate them horizontally to create a panorama
panorama_2 = cv2.warpPerspective(neighboring_image, H2to1, (img1.shape[1] + neighboring_image.shape[1], img1.shape[0]))
panorama_2[0:img1.shape[0], 0:img1.shape[1]] = img1

# Save the panorama
cv2.imwrite('../result/panorama_2.png', panorama_2)

# Use homography to warp img3 to the reference image's coordinate system
# and concatenate them horizontally to create a panorama
panorama_3 = cv2.warpPerspective(img3, H3to1, (panorama_2.shape[1] + img3.shape[1], panorama_2.shape[0]))
panorama_3[0:panorama_2.shape[0], 0:panorama_2.shape[1]] = panorama_2

# Save the panorama
cv2.imwrite('../result/panorama_3.png', panorama_3)

# Use homography to warp img4 to the reference image's coordinate system
# and concatenate them horizontally to create the final panorama
panorama_4 = cv2.warpPerspective(img4, H4to1, (panorama_3.shape[1] + img4.shape[1], panorama_3.shape[0]))
panorama_4[0:panorama_3.shape[0], 0:panorama_3.shape[1]] = panorama_3

# Save the final panorama
cv2.imwrite('../result/panorama_4.png', panorama_4)